<a href="https://colab.research.google.com/github/Matheesan2113/RandomProjects/blob/master/Spam_filter_github_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Projects in NLP Spam Filter --> Youtube. FIX REGEX CONDITIONS

In [22]:
import sys
import nltk
import sklearn
import pandas
import numpy

print('version: '+ sys.version)
print('pandas: ' + pandas.__version__)
print('numpy: ' + numpy.__version__)

version: 3.6.9 (default, Jul 17 2020, 12:50:27) 
[GCC 8.4.0]
pandas: 1.0.5
numpy: 1.18.5


## Load Dataset


In [23]:
import pandas as pd
import numpy as np

#Load dataset sms messages#
df = pd.read_table('SMSSpamCollection', header = None, encoding = "utf-8")

print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       5572 non-null   object
 1   1       5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB
None
      0                                                  1
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [24]:
# Check Class Distribution (Can affect performance of ML Algorithms, so you want to normalize this)
classes = df[0]
print(classes.value_counts())

ham     4825
spam     747
Name: 0, dtype: int64


## Pre-process Data

In [25]:
# Convert class labels to binary values
## use sklearn label encoders to change labels to binary values

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
Y = encoder.fit_transform(classes)

print(classes[:10])
print(Y[:10])

0     ham
1     ham
2    spam
3     ham
4     ham
5    spam
6     ham
7     ham
8    spam
9    spam
Name: 0, dtype: object
[0 0 1 0 0 1 0 0 1 1]


In [26]:
# Store SMS text message data

text_messages = df[1]
print(text_messages[:10])

0    Go until jurong point, crazy.. Available only ...
1                        Ok lar... Joking wif u oni...
2    Free entry in 2 a wkly comp to win FA Cup fina...
3    U dun say so early hor... U c already then say...
4    Nah I don't think he goes to usf, he lives aro...
5    FreeMsg Hey there darling it's been 3 week's n...
6    Even my brother is not like to speak with me. ...
7    As per your request 'Melle Melle (Oru Minnamin...
8    WINNER!! As a valued network customer you have...
9    Had your mobile 11 months or more? U R entitle...
Name: 1, dtype: object


In [37]:
# USE REGEX to replace emails, phone numbers, symbols, other numbers

# replace email addresses with 'emailaddr

processed = text_messages.str.replace(r'^\w+@[a-zA-Z_]+?\.[a-zA-Z]{2,3}$', 'emailaddr')

# replace webaddr with webaddr
processed = processed.str.replace(r'^((?:2[0-5]{2}|1\d{2}|[1-9]\d|[1-9])\.(?:(?:2[0-5]{2}|1\d{2}|[1-9]\d|\d)\.){2}(?:2[0-5]{2}|1\d{2}|[1-9]\d|\d)):(\d|[1-9]\d|[1-9]\d{2,3}|[1-5]\d{4}|6[0-4]\d{3}|654\d{2}|655[0-2]\d|6553[0-5])$', 'webaddress')

processed = processed.str.replace(r'$', 'moneysymb')

processed = processed.str.replace(r'^[2-9]\d{2}-\d{3}-\d{4}$', 'phonenumber')

processed = processed.str.replace(r'/\[\d+\]/g', 'number')

processed = processed.str.replace(r'[\.\,\'\(\)[]{}\!\?\\-]', '')

processed = processed.str.replace('\s+', ' ')

processed = processed.str.replace('\A\s+', '')

print(processed[:10])

0    Go until jurong point, crazy.. Available only ...
1                        Ok lar... Joking wif u oni...
2    Free entry in 2 a wkly comp to win FA Cup fina...
3    U dun say so early hor... U c already then say...
4    Nah I don't think he goes to usf, he lives aro...
5    FreeMsg Hey there darling it's been 3 week's n...
6    Even my brother is not like to speak with me. ...
7    As per your request 'Melle Melle (Oru Minnamin...
8    WINNER!! As a valued network customer you have...
9    Had your mobile 11 months or more? U R entitle...
Name: 1, dtype: object


In [38]:
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
procesed = processed.apply(lambda x: ' '.join(term for term in x.split() if term not in stop_words))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [39]:
ps = nltk.PorterStemmer()

processed = processed.apply(lambda x: ' '.join(ps.stem(term) for term in x.split()))

In [40]:
print(processed)

0       Go until jurong point, crazy.. avail onli in b...
1                             Ok lar... joke wif u oni...
2       free entri in 2 a wkli comp to win FA cup fina...
3       U dun say so earli hor... U c alreadi then say...
4       nah I don't think he goe to usf, he live aroun...
                              ...                        
5567    thi is the 2nd time we have tri 2 contact u. U...
5568                     will ü b go to esplanad fr home?
5569    pity, * wa in mood for that. so...ani other su...
5570    the guy did some bitch but I act like i'd be i...
5571                             rofl. it true to it name
Name: 1, Length: 5572, dtype: object


In [41]:
#tokenize each word
nltk.download('punkt')
from nltk.tokenize import word_tokenize

#bag of words model, extract words

all_words = []

for message in processed:
  words = word_tokenize(message)
  for w in words:
    all_words.append(w)

all_words = nltk.FreqDist(all_words)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [42]:
print('Number of Words: {}'.format(len(all_words)))
print('Most Number of Words: {}'.format(all_words.most_common(15)))

Number of Words: 9499
Most Number of Words: [('.', 4811), ('you', 2165), ('to', 2160), (',', 1939), ('?', 1550), ('I', 1480), ('!', 1397), ('i', 1363), ('a', 1339), ('the', 1333), ('...', 1261), ('and', 973), ('&', 922), ('it', 860), ('in', 832)]
